In [1]:
import rasterio
import numpy as np
from scipy import stats
import glob
import os.path
import re
from datetime import datetime
import copy

In [2]:
THRESHOLD = 6
DATE_REGEX = 'PRISM_tmean_stable_4kmD1_(\d*)_bil.bil'
FILE_REGEX = 'PRISM/**/*%s*.bil'

dayRegex = re.compile(DATE_REGEX)

In [3]:
interesting_years = range(1981, 2018, 10)

In [4]:
def thresholdDay(tmean, threshold):
    degree_days = tmean - threshold
    return degree_days

In [5]:
def readFile(file):
    with rasterio.open(file, driver='Ehdr') as src:
        tmean = src.read(1, masked=True)
        # tmean_threshold = thresholdDay(tmean, THRESHOLD)
        return (src.meta, tmean)

In [6]:
def writeFile(file, dest, meta):
    meta = copy.deepcopy(meta)
    #file = copy.deepcopy(file)
    file = np.ma.filled(file, fill_value=meta['nodata'])
    file = file.astype(meta['dtype'])

    meta['driver'] = 'GTiff'    
    
    with rasterio.open(dest, 'w', **meta) as dst:
        dst.write(file, 1)
        del meta, file

In [7]:
def thresholdYear(yearStart, yearEnd):
    aggregate = []
    meta = None
    lookupYears = list(range(yearStart, yearEnd))
    days = [day for year in lookupYears for day in glob.glob(FILE_REGEX % year)]

    print('%s-%s: Processing %s days in %s years' % (yearStart, yearEnd, len(days), len(lookupYears)))

    for year in lookupYears:
        yearAggregate = dict()
        for day in glob.glob(FILE_REGEX % year):
            basename = os.path.basename(day)
            dateString = dayRegex.search(basename).groups(0)[0]
            objDate = datetime.strptime(dateString, '%Y%m%d')
            
            if objDate.year in lookupYears:
                (tmeta, tmean) = readFile(day)
                yearAggregate[objDate] = tmean
                meta = tmeta
            else: pass
        if yearAggregate: aggregate.append(yearAggregate)
    return (aggregate, meta)

In [8]:
def remap(matrix):
    m = matrix
    m[(m > 0) & (m < 800)] = 800
    return m

In [12]:
def aggregateYears(aggregates, month, threshold):
    years_filtered = []
    for year in aggregates:
        days = [matrix - threshold for day, matrix in year.items() if day.month <= month]
        years_filtered.append(days)

    yearsComposite = [np.ma.mean(day, axis=0) for day in zip(*years_filtered)]
    #yearsComposite = [year - threshold for year in yearsComposite]
    yearsComposite = [np.ma.masked_less_equal(year, 0) for year in yearsComposite]
    yearsComposite = np.ma.sum(yearsComposite, axis=0)
    #yearsComposite = remap(yearsComposite)
    #yearsComposite = yearsComposite // 100
    #yearsComposite = yearsComposite * 100
    
    return yearsComposite

In [13]:
def processYear(year, increment=9):
    yearStart = year
    yearEnd = year + increment
    
    dest = './dist/%s-%s.tiff' % (yearStart, yearEnd)

    (aggregates, meta) = thresholdYear(yearStart, yearEnd)
    
    yearlyThreshold = aggregateYears(aggregates, 8, THRESHOLD)

    writeFile(yearlyThreshold, dest, meta)

In [14]:
for year in interesting_years:
    processYear(year, increment=9)

1981-1990: Processing 3287 days in 9 years
1991-2000: Processing 3287 days in 9 years
2001-2010: Processing 3287 days in 9 years
2011-2020: Processing 2635 days in 9 years


In [17]:
def reclassifyRaster(raster, interval):
    with rasterio.open(raster) as src:
        rasterArr = src.read(1, masked=True)
        return rasterArr

In [18]:
t = reclassifyRaster('./dist/1981-1990.tiff', 80)

In [19]:
rasterio.plot.show(t)

AttributeError: module 'rasterio' has no attribute 'plot'